In [1]:
from swissisoform.alternative_isoforms import AlternativeIsoform
from swissisoform.utils import validate_ensembl_mappings, print_validation_results, update_bed_with_reference_names, print_update_results

In [5]:
results = validate_ensembl_mappings(
    "../data/ribosome_profiling/RiboTISHV6_Ly2024_AnnoToTruncation_exonintersect.bed"
)
print_validation_results(results)

Fetching current Ensembl reference data...
Retrieved 48379 reference mappings

Validation Results:
Total entries processed: 4926
Successfully validated: 4922

ENSEMBL IDs not found in reference (4 found):
Line 426: ENSG00000112096 (SOD2)
Line 427: ENSG00000112096 (SOD2)
Line 1894: ENSG00000215271 (HOMEZ)
Line 1895: ENSG00000215271 (HOMEZ)

Reference mismatches (154 found):
Line 45: ENSG00000065427
  BED gene name: KARS
  Reference gene name: KARS1

Line 87: ENSG00000141699
  BED gene name: FAM134C
  Reference gene name: RETREG3

Line 104: ENSG00000119333
  BED gene name: WDR34
  Reference gene name: DYNC2I2

Line 109: ENSG00000112941
  BED gene name: PAPD7
  Reference gene name: TENT4A

Line 113: ENSG00000109685
  BED gene name: WHSC1
  Reference gene name: NSD2

Line 167: ENSG00000146729
  BED gene name: GBAS
  Reference gene name: NIPSNAP2

Line 202: ENSG00000157870
  BED gene name: FAM213B
  Reference gene name: PRXL2B

Line 203: ENSG00000106105
  BED gene name: GARS
  Reference gen

In [7]:
stats = update_bed_with_reference_names(
    "../data/ribosome_profiling/RiboTISHV6_Ly2024_AnnoToTruncation_exonintersect.bed",
    "../data/ribosome_profiling/RiboTISHV6_MD2025_AnnoToTruncation_exonintersect.bed",
)
print_update_results(stats)

Fetching current Ensembl reference data...
Retrieved 48379 reference mappings

Update Results:
Total entries: 4926
Successfully updated: 4922
Invalid format entries: 0
ENSEMBL IDs not found: 4
Gene names that were different: 154


In [8]:
# Initialize handlers
alt_isoforms = AlternativeIsoform()

# Load BED file
alt_isoforms.load_bed(
    "../data/ribosome_profiling/RiboTISHV6_MD2025_AnnoToTruncation_exonintersect.bed"
)

gene_list = alt_isoforms.get_gene_list()

In [9]:
# Write gene_list to a txt file in the ribosome profiling directory
with open("../data/ribosome_profiling/gene_list.txt", "w") as f:
    for gene in gene_list:
        f.write(gene + "\n")